In [20]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
from sklearn.preprocessing import MinMaxScaler
import random
import implicit

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of item,user index into list

    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of item-user pairs without replacement

    item_inds = [index[0] for index in samples] # Get the item row indices

    user_inds = [index[1] for index in samples] # Get the user column indices

    
    training_set[item_inds, user_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    mask = {}
    #reference dictionary
    
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user columns that were altered

## Data loading and preprocessing

In [21]:
#load the rsvp, member and events data 
rsvp = pd.read_csv('rsvp.csv')
member = pd.read_csv('members.csv',encoding='latin-1')
events = pd.read_csv('events.csv',encoding='latin-1')

In [22]:
#preprocesssing
main_data = pd.merge(rsvp, events, on= 'event_id', how='inner') ; main_data.head()
temp_data = main_data[['event_id', 'member_id', 'group_id', 'group.name']] ; temp_data.head()
main_data_2 =temp_data.groupby(['group_id','member_id']).size().reset_index() ; print (main_data_2.head())
print (main_data_2.shape) ; print (main_data_2.member_id.nunique())
main_data_2.columns = ['group_id', 'member_id','rsvp_sum']

   group_id  member_id   0
0    413427    5033164   1
1    413427   12142571   1
2    413427   31492942  12
3    413427   36962882   1
4    413427   51417262   1
(3039, 3)
2958


In [23]:
main_data.head(2)

,event_id,member_id,created,description,duration,event_url,fee.accepts,fee.amount,fee.currency,fee.description,fee.label,fee.required,group.created,group.group_lat,group.group_lon,group_id,group.join_mode,group.name,group.urlname,group.who,headcount,how_to_find_us,maybe_rsvp_count,event_name,photo_url,rating.average,rating.count,rsvp_limit,event_status,event_time,updated,utc_offset,venue.address_1,venue.address_2,venue.city,venue.country,venue_id,venue.lat,venue.localized_country_name,venue.lon,venue.name,venue.phone,venue.repinned,venue.state,venue.zip,visibility,waitlist_count,why,yes_rsvp_count
0,cqkdxnywnbpc,12067672,2017-10-25 21:01:19,~,10800,https://www.meetup.com/meetup-group-kZjmZIoP/e...,others,0.0,not_found,per person,Price,0,2017-10-12 18:57:18,37.779999,-122.419998,26243452,open,Reading Chinese Books,meetup-group-kZjmZIoP,Members,0,"The lobby is with full glass walls,we can see ...",0,NaN,not_found,0.0,-1,15,upcoming,2017-11-01 02:30:00,2017-10-25 21:01:19,-25200,1222 Harrison Street,not_found,san francisco,us,25548384,37.773609,USA,-122.409248,L Seven Apartments Lobby,-1,0,CA,-1,public,0,not_found,1
1,cqkdxnywnbpc,234733619,2017-10-25 21:01:19,~,10800,https://www.meetup.com/meetup-group-kZjmZIoP/e...,others,0.0,not_found,per person,Price,0,2017-10-12 18:57:18,37.779999,-122.419998,26243452,open,Reading Chinese Books,meetup-group-kZjmZIoP,Members,0,"The lobby is with full glass walls,we can see ...",0,NaN,not_found,0.0,-1,15,upcoming,2017-11-01 02:30:00,2017-10-25 21:01:19,-25200,1222 Harrison Street,not_found,san francisco,us,25548384,37.773609,USA,-122.409248,L Seven Apartments Lobby,-1,0,CA,-1,public,0,not_found,1


In [24]:
#standardizing to 1 - 10 scale 
main_data_2['rsvp_total'] = 1 + (main_data_2['rsvp_sum']-main_data_2['rsvp_sum'].min())*(10-1)/(main_data_2['rsvp_sum'].max()-main_data_2['rsvp_sum'].min())

In [25]:
data = main_data_2.dropna()
data = data.copy()

# Converting the numbers to categories to be used for creating the categorical codes to avoid using long hash keys 
data['member_id'] = data['member_id'].astype('category')
data['group_id'] = data['group_id'].astype('category')

#cat.codes creates a categorical id for the users and artists
data['user_id'] = data['member_id'].cat.codes
data['grp_id'] = data['group_id'].cat.codes

# The implicit library expects data as a item-user matrix so we
# create two matrices, one for fitting the model (item-user) 
# and one for recommendations (user-item)

sparse_item_user = sparse.csr_matrix((data['rsvp_total'].astype(float), (data['grp_id'], data['user_id'])))
sparse_user_item = sparse.csr_matrix((data['rsvp_total'].astype(float), (data['user_id'], data['grp_id'])))

In [26]:
matrix_size = sparse_user_item.shape[0]*sparse_user_item.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_user_item.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

97.81407794225541

In [27]:
# 20% of the data has been masked for this exercise
product_train, product_test, product_users_altered = make_train(sparse_item_user, pct_test = 0.2)

In [28]:
# Initialize the als model and fit it using the sparse item-user matrix
# Parameters that we have chosen
# 1. factors = 20 -- Latent factors for user and item vectors
# 2. iterations = 20 -- Number of iterations to use while fitting the data
# 3. regularization = 0.1 -- regularization constant to be used in the cost function

model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=40)

# Calculate the confidence by multiplying it by our alpha value.(alpha value corresponds to the confidence metric 
# that we discussed earlier)

alpha_val = 15
data_conf = (product_train * alpha_val).astype('double')

# We have used an alpha_val of 15 after performing some iterations with different alpha values
#Fit the model
model.fit(data_conf)

In [29]:
item_vecs = model.item_factors
user_vecs = model.user_factors

print('Shape of group vector matrix : ', item_vecs.shape)
print('Shape of member vector matrix : ', user_vecs.shape)

Shape of group vector matrix :  (47, 20)
Shape of member vector matrix :  (2958, 20)


In [30]:
from sklearn import metrics
import matplotlib.pylab as plt
def auc_score(predictions, test):
    '''
    This simple function will output the area under the curve using sklearn's metrics. 
    
    parameters:
    
    - predictions: your prediction output
    
    - test: the actual target result you are comparing to
    
    returns:
    
    - AUC (area under the Receiver Operating Characterisic curve)
    '''
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

def calc_mean_auc(training_set, altered_users, predictions, test_set):
    '''
    This function will calculate the mean AUC by user for any user that had their user-item matrix altered. 
    
    parameters:
    
    training_set - The training set resulting from make_train, where a certain percentage of the original
    user/item interactions are reset to zero to hide them from the model 
    
    predictions - The matrix of your predicted ratings for each user/item pair as output from the implicit MF.
    These should be stored in a list, with user vectors as item zero and item vectors as item one. 
    
    altered_users - The indices of the users where at least one user/item pair was altered from make_train function
    
    test_set - The test set constucted earlier from make_train function
    
    
    returns:
    
    The mean AUC (area under the Receiver Operator Characteristic curve) of the test set only on user-item interactions
    there were originally zero to test ranking ability in addition to the most popular items as a benchmark.
    '''
    
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_items = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    item_vecs = predictions[1]
    for user in altered_users: # Iterate through each user that had an item altered
        training_column = training_set[:,user].toarray().reshape(-1) # Get the training set column
        zero_inds = np.where(training_column == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        user_vec = predictions[0][user,:]
        pred = user_vec.dot(item_vecs).toarray()[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,user].toarray()[zero_inds,0].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_items[zero_inds] # Get the item popularity for our chosen items
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))  
   # Return the mean AUC rounded to three decimal places for both test and popularity benchmark

calc_mean_auc(product_train, product_users_altered,
              [sparse.csr_matrix(user_vecs), sparse.csr_matrix(item_vecs.T)], product_test)
# AUC for our recommender system

(0.51, 0.839)

In [31]:
test=pd.merge(data, events[['group.name','group_id']], on = 'group_id', how = 'inner')

In [32]:
test=test.drop_duplicates()

In [33]:
# raeding chinese books group - 26243452

# Find the 10 most similar to red hot chilli peppers
group_id = 30
n_similar = 10 # getting the top ten similar items
print ("------------Actual group-------------------")
print (test[test['grp_id'] == group_id]['group.name'])
print ("------------Actual group-------------------")
# Use implicit to get similar items.
similar = model.similar_items(group_id, 3)
# Print the names of our most similar artists
for group in similar:
    idx, score = group
    print (test['group.name'].loc[test.grp_id == idx].iloc[0])

------------Actual group-------------------
97011    Learn-Investing-Concepts-With-Cashflow-101
Name: group.name, dtype: object
------------Actual group-------------------
Learn-Investing-Concepts-With-Cashflow-101
Mon/Thur Night Improv Classes
Womens Prosperity Network - NYC


### Generating recommendations

In [34]:
test.head()

,group_id,member_id,rsvp_sum,rsvp_total,user_id,grp_id,group.name
0,413427,5033164,1,1.000000,82,0,NYC Service Businesses & Entrepreneurs Associa...
13,413427,12142571,1,1.000000,294,0,NYC Service Businesses & Entrepreneurs Associa...
26,413427,31492942,12,3.020408,468,0,NYC Service Businesses & Entrepreneurs Associa...
39,413427,36962882,1,1.000000,496,0,NYC Service Businesses & Entrepreneurs Associa...
52,413427,51417262,1,1.000000,556,0,NYC Service Businesses & Entrepreneurs Associa...


In [54]:
#top member
test['rank'] = test.rsvp_total.rank(ascending = False, method='max') 
test_1 = test[test['rank'] <10]
test_1.head(25)

,group_id,member_id,rsvp_sum,rsvp_total,user_id,grp_id,group.name,rank
32159,2148441,5189189,42,8.530612,86,17,SF Free School,3.0
43415,2148441,88770122,39,7.979592,719,17,SF Free School,5.0
108186,18177413,192517329,33,6.877551,1366,35,Bryant Park Toastmasters,7.0
108286,18177413,195703963,50,10.000000,1429,35,Bryant Park Toastmasters,1.0
108786,18177413,203392611,31,6.510204,1600,35,Bryant Park Toastmasters,9.0
109386,18177413,224325846,45,9.081633,1940,35,Bryant Park Toastmasters,2.0
114605,18468560,184845608,33,6.877551,1187,40,Lexington/Vanderbilt toastmasters public speak...,7.0
120420,26243452,78357452,31,6.510204,675,46,Reading Chinese Books,9.0
121470,26243452,234733619,39,7.979592,2205,46,Reading Chinese Books,5.0


In [57]:
# user id is 1429 
user_id = 40

# Use the implicit recommender.
recommended = model.recommend(user_id, sparse_user_item,N = 10,filter_already_liked_items = False)

artists = []
scores = []

# Get artist names from ids
for item in recommended:
    idx, score = item
    artists.append(test['group.name'].loc[test.grp_id == idx].iloc[0])
    scores.append(score)

# Create a dataframe of artist names and scores
recommendations = pd.DataFrame({'artist': artists, 'score': scores})

print (recommendations)

                                              artist     score
0                                   Smutty Book Club  0.976159
1                            Empower Your Creativity  0.394143
2  The Professional Womens Network of San Francis...  0.188454
3  NYC Contemporary Art Lovers, Promoters & Oppor...  0.186421
4            Entrepreneur Mastery Mentoring - Meetup  0.166089
5  NYC Service Businesses & Entrepreneurs Associa...  0.136232
6  Lexington/Vanderbilt toastmasters public speak...  0.114514
7                Marketing, Advertising and PR Group  0.103782
8                        Learn Website Marketing NYC  0.101414
9                             Figure Drawing Chicago  0.092845
